# Evaluation Notebook — Course Recommendation FastAPI (Chroma RAG)

This notebook hits your running FastAPI service and records the **execution responses** for a fixed set of evaluation queries. It:

- Checks service health
- Submits the five evaluation queries
- Prints and tabulates results (raw Chroma retrieval + final LLM re-ranked recommendations)
- Saves a timestamped JSON/CSV bundle to `evaluation_runs/`

> **Prerequisites**
> 1. Start the API in a terminal (same repo where `app.py` lives):
>    ```bash
>    uvicorn app:app --host 0.0.0.0 --port 8000 --reload
>    ```
> 2. Ensure `GOOGLE_API_KEY` is set in your environment (or `.env`) because the pipeline **always uses the LLM** for structured output.  
>    If the key is missing, `/recommend` will fail with HTTP 400 (and internally the pipeline would still require the key).  
> 3. First start will auto-download the catalog CSV and build the Chroma index. (Make sure the machine has internet access.)

> **Notes**
> - API endpoints used: `/health` and `/recommend` (POST). citeturn1search1  
> - The app builds/persists a Chroma index at `storage/chroma` and fetches dataset from a GitHub raw CSV on startup. citeturn1search1  
> - Request body keys: `query`, `top_k`, `use_llm_format`; server expects `use_llm_format=True` and a valid Google API key. citeturn1search1


## Evaluation Queries

We will execute the following five queries **exactly as provided**:

1. “I’ve completed the ‘Python Programming for Data Science’ course and enjoy data visualization. What should I take next?”
2. “I know Azure basics and want to manage containers and build CI/CD pipelines. Recommend courses.”
3. “My background is in ML fundamentals; I’d like to specialize in neural networks and production workflows.”
4. “I want to learn to build and deploy microservices with Kubernetes—what courses fit best?”
5. “I’m interested in blockchain and smart contracts but have no prior experience. Which courses do you suggest?”


## Setup
Install/verify dependencies (if needed) and set the API base URL.

> Tip: If you're inside a clean environment, you might need:
> ```bash
> pip install requests pandas matplotlib
> ```


In [1]:
# Configuration — edit BASE_URL if your server runs elsewhere
BASE_URL = "http://localhost:8000"  # FastAPI server
TOP_K = 5
USE_LLM_FORMAT = True  # the pipeline uses LLM-structured outputs anyway

import os
print("GOOGLE_API_KEY set:", bool(os.getenv("GOOGLE_API_KEY")))


GOOGLE_API_KEY set: True


In [2]:
# Health check
import requests
try:
    r = requests.get(f"{BASE_URL}/health", timeout=30)
    print("Status:", r.status_code)
    print("Payload:", r.json())
except Exception as e:
    raise SystemExit(f"Health check failed — is the API running at {BASE_URL}?{e}")

Status: 200
Payload: {'status': 'ok'}


In [3]:
# Helper: post a single query to /recommend and return parsed result
import requests
import json

def run_query(query: str, top_k: int = TOP_K, use_llm_format: bool = USE_LLM_FORMAT):
    payload = {
        "query": query,
        "top_k": int(top_k),
        "use_llm_format": bool(use_llm_format),
    }
    url = f"{BASE_URL}/recommend"
    r = requests.post(url, json=payload, timeout=120)
    try:
        r.raise_for_status()
    except Exception:
        print("Response text:", r.text)
        raise
    data = r.json()
    # Server returns: {"result": {"profile": ..., "raw": ..., "llm_json": ...}}
    return data["result"]


In [4]:
# Evaluation query set
queries = [
    "I’ve completed the ‘Python Programming for Data Science’ course and enjoy data visualization. What should I take next?",
    "I know Azure basics and want to manage containers and build CI/CD pipelines. Recommend courses.",
    "My background is in ML fundamentals; I’d like to specialize in neural networks and production workflows.",
    "I want to learn to build and deploy microservices with Kubernetes—what courses fit best?",
    "I’m interested in blockchain and smart contracts but have no prior experience. Which courses do you suggest?",
]
print(f"Loaded {len(queries)} queries.")


Loaded 5 queries.


In [5]:
# Execute all queries and collect results
from pprint import pprint
import pandas as pd

results = []
for idx, q in enumerate(queries, start=1):
    print("=== Query", idx, "===")
    print(q)
    res = run_query(q)
    results.append({"query": q, **res})
    # Pretty print profile
    print("— Extracted Profile —")
    pprint(res.get("profile"))
    # Show raw Chroma retrieval (baseline relevance)
    print("— Raw Retrieval (Chroma) —")
    df_raw = pd.DataFrame(res.get("raw", []))
    display(df_raw)
    # Show final LLM recommendations
    print("— LLM Recommendations —")
    df_llm = pd.DataFrame(res.get("llm_json", []))
    display(df_llm)


=== Query 1 ===
I’ve completed the ‘Python Programming for Data Science’ course and enjoy data visualization. What should I take next?
— Extracted Profile —
{'completed_courses': ['Python Programming for Data Science'],
 'interests': 'data visualization'}
— Raw Retrieval (Chroma) —


,course_id,title,description,relevance
0,C016,Python Programming for Data Science,Learn Python fundamentals for data science: va...,0.5392
1,C017,R Programming and Statistical Analysis,Get introduced to R for statistical computing ...,0.0227
2,C001,Foundations of Machine Learning,Understand foundational machine learning algor...,0.0000
3,C014,Data Visualization with Tableau,Transform raw data into compelling visual stor...,0.0000
4,C011,Big Data Analytics with Spark,Process and analyze large datasets using Apach...,0.0000


— LLM Recommendations —


,course_id,title,score,reason
0,C014,Data Visualization with Tableau,0.65,This course directly addresses your interest i...
1,C017,R Programming and Statistical Analysis,0.37,This course introduces R for statistical analy...
2,C001,Foundations of Machine Learning,0.35,This course builds upon your Python foundation...
3,C011,Big Data Analytics with Spark,0.30,This course leverages your Python knowledge to...
4,C016,Python Programming for Data Science,0.00,This course has already been completed by the ...


=== Query 2 ===
I know Azure basics and want to manage containers and build CI/CD pipelines. Recommend courses.
— Extracted Profile —
{'completed_courses': ['Azure basics'],
 'interests': 'container management, CI/CD pipelines'}
— Raw Retrieval (Chroma) —


...

,course_id,title,score,reason
0,C009,Containerization with Docker and Kubernetes,0.91,This course directly addresses your interest i...
1,C010,APIs and Microservices Architecture,0.45,This course is highly relevant as it focuses o...
2,C007,Cloud Computing with Azure,0.35,This course introduces cloud deployment with A...
3,C025,MLOps: Productionizing Machine Learning,0.10,"While focused on ML, this course includes rele..."
4,C002,Deep Learning with TensorFlow and Keras,0.00,This course is not directly aligned with your ...


=== Query 5 ===
I’m interested in blockchain and smart contracts but have no prior experience. Which courses do you suggest?
— Extracted Profile —
{'completed_courses': [], 'interests': 'blockchain and smart contracts'}
— Raw Retrieval (Chroma) —


,course_id,title,description,relevance
0,C023,Blockchain Technology and Smart Contracts,Understand blockchain fundamentals: cryptograp...,0.2764
1,C021,Cybersecurity Fundamentals,Get introduced to cybersecurity principles: th...,0.0000
2,C019,Agile and Scrum Mastery,Adopt Agile frameworks to enhance team product...,0.0000
3,C001,Foundations of Machine Learning,Understand foundational machine learning algor...,0.0000
4,C010,APIs and Microservices Architecture,Design and implement RESTful and GraphQL APIs ...,0.0000


— LLM Recommendations —


,course_id,title,score,reason
0,C023,Blockchain Technology and Smart Contracts,0.63,This course directly aligns with your strong i...
1,C021,Cybersecurity Fundamentals,0.05,Cybersecurity fundamentals provide a valuable ...
2,C019,Agile and Scrum Mastery,0.00,This course focuses on project management meth...
3,C001,Foundations of Machine Learning,0.00,This course covers machine learning fundamenta...
4,C010,APIs and Microservices Architecture,0.05,Understanding APIs and microservices can be be...


In [6]:
# Save a timestamped artifact of this run (JSON + per-query CSVs)
import os, json, time
import pandas as pd

run_ts = time.strftime("%Y%m%d_%H%M%S")
OUT_DIR = os.path.join("evaluation_runs", run_ts)
os.makedirs(OUT_DIR, exist_ok=True)

# Save the full JSON
with open(os.path.join(OUT_DIR, "full_results.json"), "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

# Save per-query CSVs
for i, res in enumerate(results, start=1):
    df_raw = pd.DataFrame(res.get("raw", []))
    df_llm = pd.DataFrame(res.get("llm_json", []))
    df_raw.to_csv(os.path.join(OUT_DIR, f"q{i:02d}_raw.csv"), index=False)
    df_llm.to_csv(os.path.join(OUT_DIR, f"q{i:02d}_llm.csv"), index=False)

print("Saved artifacts to:", OUT_DIR)


Saved artifacts to: evaluation_runs\20251127_111945


In [ ]:
# # Optional: quick visualization of scores per query
# import matplotlib.pyplot as plt
# import pandas as pd

# for i, res in enumerate(results, start=1):
#     df = pd.DataFrame(res.get("llm_json", []))
#     if df.empty or "score" not in df.columns:
#         continue
#     plt.figure(figsize=(8, 4))
#     plt.bar(df["title"], df["score"], color="#4c78a8")
#     plt.title(f"Query {i}: Final Recommendation Scores")
#     plt.ylabel("Score (0–1)")
#     plt.xticks(rotation=45, ha="right")
#     plt.tight_layout()
#     plt.show()
